In [2]:

import requests
import json

import pandas as pd
import numpy as np

import pickle

import sqlite3





# 1) FUNCTIONS

In [3]:
## FUNCTIONS

def get_historic_data(interval:str,range:str,symbols:list):
    assert interval in ('1m','5m','15m','1d','1wk','1mo')
    # assert range    in ('1d','5d','1mo','3mo','6mo','1y','5y','max',None)

    url = "https://yfapi.net/v8/finance/spark"
    with open('../yahoo_api_keys.txt') as f:
        api_key = {'x-api-key': f.read().rstrip()}


    querystring = {'interval': interval, 'range': range, 'symbols':','.join(symbols)}


    response = requests.request("GET", url, headers=api_key, params=querystring)

    return response, querystring


def clean_response(request_response):
    ret_json = json.loads(request_response.text)

    ret_df = None
    for key in ret_json.keys():
        print(key)
        if ret_df is None:
            ret_df = pd.json_normalize(ret_json[key])
        else:
            ret_df = pd.concat([pd.json_normalize(ret_json[key]),ret_df])
        

    ret_df[['timestamp','end','start']] = ret_df[['timestamp','end','start']].applymap(lambda x: pd.to_datetime(x, unit='s'))

    ret_df['series'] = ret_df.apply(lambda x: [pd.Series(x.close, index=x.timestamp,name=x.symbol)],axis=1)

    ret_df = ret_df.drop(['timestamp','close'],axis=1)

    symb = list(ret_df.symbol)

    out = pd.DataFrame({'Timestamp': ret_df.iloc[0].series[0].index,'Ticker': ret_df.iloc[0].series[0].name, 'Close': ret_df.iloc[0].series[0].values})
    if out.Timestamp.iloc[-1].second != 00:             # delete uneven values
        out = out[:-1]

    for i in range(1,len(symb)):
        out = pd.concat([out, pd.DataFrame({'Timestamp': ret_df.iloc[i].series[0].index,'Ticker': ret_df.iloc[i].series[0].name, 'Close': ret_df.iloc[i].series[0].values})],ignore_index=True)
        if out.Timestamp.iloc[-1].second != 00:         # delete uneven values
            out = out[:-1]

    return out


def request_logger(querystring, start, end):
    with open("db/logs_query.txt", "a") as f:
        for symbol in querystring['symbols'].split(','):
            
            # start_date_req = str(querystring['start_date'])
            # end_date_req   = str(querystring['end_date']) 
            
            f.write(symbol +'|'+ str(start) +'|'+ str(end) +'|'+ querystring['interval'] +'\n')     #+'|'+ start_date_req +'|'+ end_date_req 
    f.close()


In [4]:
def data_availability(inp_ticker,inp_interval,inp_start,inp_end):

    result_out = [0,[]]

    inp_start = pd.to_datetime(inp_start)
    inp_end   = pd.to_datetime(inp_end)
    
    if inp_interval == '5m':
        interval_conv = '5min'
    elif inp_interval == '15m':
        interval_conv = '15min'
    elif inp_interval == '1d':
        interval_conv = 'D'
    elif inp_interval == '1m':
        interval_conv = '1min'

    data = pd.read_csv('db/logs_query.txt', sep="|", header=None)
    data.columns = ['Ticker','Start','End','Interval']
    data = data.drop_duplicates(subset=['Ticker', 'Start', 'End', 'Interval']).reset_index(drop=True)

    interval_rank = ['1m','5m','15m','1d','1wk','1mo']
    # Range = namedtuple('Range', ['start', 'end'])

    if data.Ticker.isin([inp_ticker]).sum() == 0:               # Exit early when ticker is not available
        result_out[1] = pd.date_range(inp_start,inp_end, freq=interval_conv)
        return result_out

    data = data.loc[(data.Ticker == inp_ticker) & (data.Interval.isin(interval_rank[:-interval_rank.index(inp_interval)]))].drop_duplicates(['Start','End'])

    ticker_range = pd.date_range(inp_start,inp_end, freq=interval_conv)


    for index, row in data.iterrows():
        data_range = pd.date_range(pd.to_datetime(row.Start),pd.to_datetime(row.End), freq=interval_conv)

        ticker_range = ticker_range.difference(data_range)
        if ticker_range.size == 0:
            result_out[0] = 1
            break

    result_out[1] = ticker_range

    return result_out


def sql_string(ticker,interval,start,end,table='stocks'):


    if interval != '1m':
        if interval == '5m':
            interval_fct = 'M'
            interval_seq = ('00','05','10','15','20','25','30','35','40','45','50')
        if interval == '15m':
            interval_fct = 'M'
            interval_seq = ('00','15','30','45')   
        if interval == '1d':
            interval_fct = '%H:%M'
            interval_seq = ('14:30')
        query = f"SELECT * FROM '{table}' WHERE Ticker= '{ticker}' AND Timestamp >= '{start}' AND Timestamp <= '{end}' AND strftime('%{interval_fct}', Timestamp) IN {interval_seq} ORDER BY Timestamp"


    else:
        query = f"SELECT * FROM '{table}' WHERE Ticker= '{ticker}' AND Timestamp >= '{start}' AND Timestamp <= '{end}' ORDER BY Timestamp"    
    
    return query


def pull_data(ticker:str, interval:str, start_time:str, end_time:str):

    con = sqlite3.connect('db\hist_data.db')

    data_avail = data_availability(ticker, interval, start_time,end_time)
    complete = True

    # Full avalability
    if data_avail[0] == 1:
        query = sql_string(ticker,interval,start_time,end_time)
        print('All data available in database.')
        return pd.read_sql_query(query, con), complete

    # Partial Match: only for a single empty window for now. Fetch as much data as possible
    if data_avail[0] == 0:
        
        if interval in ('5m','15m'):
            query_range = '60d'
            if (pd.Timedelta(data_avail[1][-1]-data_avail[1][0],unit='day').days+1) >=60:
                complete = False
        elif interval == '1m':
            query_range = '7d'
            if (pd.Timedelta(data_avail[1][-1]-data_avail[1][0],unit='day').days+1) >=7:
                complete = False

        elif interval == '1d':
            query_range = str(pd.Timedelta(data_avail[1][-1]-data_avail[1][0],unit='day').days+1)+'d'
            complete = True

        results_raw_tmp, api_query_string = get_historic_data(interval,query_range,[ticker])
        data_clean = clean_response(results_raw_tmp)
        request_logger(api_query_string, data_clean['Timestamp'].min(), data_clean['Timestamp'].max())
        
        print('Data added to database.',query_range,'from:',data_clean['Timestamp'].min(),'   to:',data_clean['Timestamp'].max())                      # PRINT. Not correct if NO overlap...

        
        data_clean.to_sql('stocks', con, if_exists='replace', index=False)               
        
        query = sql_string(ticker,interval,start_time,end_time)
        return pd.read_sql_query(query, con), complete

# 2) APPLICATION

In [19]:
## 1) GENERAL: FILL DATABASE

# results_raw_tmp, api_query_string = get_historic_data('5m','1mo',['AAPL','MSFT', 'AMZN'], None, None)

# data_clean = clean_response(results_raw_tmp)
# request_logger(api_query_string, data_clean['Timestamp'].min(), data_clean['Timestamp'].max())

# con = sqlite3.connect('hist_data.db')
# cur = con.cursor()
# data_clean.to_sql('stocks', con, if_exists='replace', index=False)
# con.close()

In [38]:
## 2) SPECIFIC: REQUEST VALUES; FILL DATABASE IF NOT EXISTING


# data_request, complete = pull_data('MSFT','15m','2022-02-01 14:30:00','2022-03-07 14:30:00')
data_request, complete = pull_data('TSLA','1m','2022-02-01 14:30:00','2022-03-06 14:30:00')

if complete==False:
    print('WARNING: Data incomplete')

data_request


TSLA
Data added to database. 7d from: 2022-02-28 14:30:00    to: 2022-03-08 16:31:00


,Timestamp,Ticker,Close
0,2022-02-28 14:30:00,TSLA,822.26
1,2022-02-28 14:31:00,TSLA,818.58
2,2022-02-28 14:32:00,TSLA,823.40
3,2022-02-28 14:33:00,TSLA,826.73
4,2022-02-28 14:34:00,TSLA,829.28
...,...,...,...
1943,2022-03-04 20:55:00,TSLA,838.28
1944,2022-03-04 20:56:00,TSLA,837.81
1945,2022-03-04 20:57:00,TSLA,838.30
1946,2022-03-04 20:58:00,TSLA,838.41


In [5]:
## 3) REQUEST DATA DIRECTLY

con = sqlite3.connect('db\hist_data.db')
cur = con.cursor()

query = """
SELECT * FROM stocks
WHERE Ticker='TSLA'

ORDER BY Timestamp
"""


df_test_2 = pd.read_sql_query(query, con)
df_test_2
con.close()

,Timestamp,Ticker,Close
0,2022-02-28 14:30:00,TSLA,822.260
1,2022-02-28 14:31:00,TSLA,818.580
2,2022-02-28 14:32:00,TSLA,823.400
3,2022-02-28 14:33:00,TSLA,826.730
4,2022-02-28 14:34:00,TSLA,829.280
...,...,...,...
2455,2022-03-08 16:27:00,TSLA,813.880
2456,2022-03-08 16:28:00,TSLA,813.128
2457,2022-03-08 16:29:00,TSLA,813.760
2458,2022-03-08 16:30:00,TSLA,813.110


In [35]:
with open('data/symbols_stocks', 'rb') as fp:   # Unpickling
    symbols_stocks = pickle.load(fp)
    fp.close()

with open('data/symbols_crypto', 'rb') as fp:   # Unpickling
    symbols_crypto = pickle.load(fp)
    fp.close()

# symbols_crypto

['BTC-USD',
 'ETH-USD',
 'USDT-USD',
 'BNB-USD',
 'USDC-USD',
 'XRP-USD',
 'LUNA1-USD',
 'ADA-USD',
 'SOL-USD',
 'HEX-USD',
 'AVAX-USD',
 'BUSD-USD',
 'DOT-USD',
 'DOGE-USD',
 'UST-USD',
 'SHIB-USD',
 'MATIC-USD',
 'WBTC-USD',
 'CRO-USD',
 'DAI-USD',
 'ATOM-USD',
 'LTC-USD',
 'NEAR-USD',
 'LINK-USD',
 'WTRX-USD',
 'TRX-USD',
 'UNI1-USD',
 'FTT-USD',
 'BCH-USD',
 'STETH-USD',
 'LEO-USD',
 'ALGO-USD',
 'MANA-USD',
 'XLM-USD',
 'HBAR-USD',
 'BTCB-USD',
 'ETC-USD',
 'ICP-USD',
 'FTM-USD',
 'SAND-USD',
 'FIL-USD',
 'VET-USD',
 'AXS-USD',
 'KLAY-USD',
 'EGLD-USD',
 'FRAX-USD',
 'XMR-USD',
 'OSMO-USD',
 'THETA-USD',
 'XTZ-USD',
 'HNT-USD',
 'WAVES-USD',
 'TONCOIN-USD',
 'MIOTA-USD',
 'EOS-USD',
 'FLOW-USD',
 'BTT-USD',
 'WBNB-USD',
 'MKR-USD',
 'BTT2-USD',
 'CAKE-USD',
 'AAVE-USD',
 'GRT1-USD',
 'ONE1-USD',
 'HBTC-USD',
 'ZEC-USD',
 'GALA-USD',
 'RUNE-USD',
 'BSV-USD',
 'NEO-USD',
 'TUSD-USD',
 'KCS-USD',
 'HT-USD',
 'STX-USD',
 'QNT-USD',
 'XEC-USD',
 'CHZ-USD',
 'CELO-USD',
 'ENJ-USD',
 'YO